In [42]:
import pandas as pd
from datetime import datetime
import numpy as np

## Merge Metacritic &  jv & Gk

In [43]:
df_games = pd.read_csv('../data/01_GameSpy_Scraping_Metacritic.csv',sep=',', index_col='Unnamed: 0')
df_jv_gk = pd.read_csv('../data/02_GameSpy_Scraping_merge_gk_jv.csv',sep=';',encoding='latin-1')

In [44]:
df_jv_gk = df_jv_gk.drop(columns=['Column1'])
df_jv_gk['join']= df_jv_gk['Platform'] + df_jv_gk['Name_alphanum']
df_jv_gk = df_jv_gk.drop_duplicates(keep='first')

In [45]:
#Merging
df_merge = df_games.merge(df_jv_gk,on='join',how='right').drop_duplicates()
df_merge = df_merge.drop_duplicates(keep='first')
df_merge = df_merge.drop(columns=['NameFormated','join','JV_Name','JV_plateforms','Name_alphanum','JV_Name_alphanum'])
df_merge = df_merge.rename(columns={
    'Date':'MC_Date',
    'Metascore':'MC_Score',
    'Studio':'MC_Studio',
    'Userscore':'MC_Userscore'
})

### Traitement du dataset final

In [46]:
"""
Toutes les notes Metacritic ou JV ou GK à zéro sont converti en nan
"""
columns = ['MC_Score','MC_Userscore','JV_journalist_rate','GK_noteGK','JV_players_rates','GK_noteUsers','MC_Userscore']

for col in columns:
    df_merge[col] = df_merge[col].replace(0, np.nan).replace('-', np.nan).astype('float')

In [47]:
"""
Transformation des reviews de jv
"""
reviews_jv = []
for a in df_merge['JV_reviews']:
    rev = str(a).strip(' avis').replace(' ', '')
    reviews_jv.append(rev)

df_merge['JV_reviews'] = reviews_jv
df_merge['JV_reviews'] = df_merge['JV_reviews'] .astype('float')

In [48]:
"""
Ensemble de fonction pour l'application de la gestion des priorités entre les données Metacritic, Gamkult et jeux-vidéos.com
"""
def __Choose3ValuesNumber(Series1,Series2,Series3):
    """
    J'utilise cette fonction pour selectionner une valeur parmi trois valeurs à tester 
    """
    values_ = []

    for x, y, z in zip(Series1,Series2,Series3):
            s = pd.Series([x,y,z])
            val = round(s.mean(),2)
            values_.append(val)
        
    return values_

def __Choose2ValuesNumber(Series1,Series2):
    """
    J'utilise cette fonction pour selectionner une valeur parmi trois valeurs à tester 
    """
    values_ = []

    for x, y in zip(Series1,Series2):
            s = pd.Series([x,y])
            val = round(s.mean(),2)
            values_.append(val)
        
    return values_


def __Choose2ValuesText(Series1,Series2):
    """
    J'utilise cette fonction pour selectionner une valeur parmi deux valeurs à tester 
    """
    values_ = []

    for x, y in zip(Series1,Series2):
        if pd.isna(x) & pd.isna(y):
            val = np.nan
        elif pd.isna(x)==False & pd.isna(y)==False:
            val = x
        elif pd.isna(x)==False & pd.isna(y):
            val = x
        elif pd.isna(x) & pd.isna(y)==False:
             val = y
        values_.append(val)
        
    return values_
def __Choose3ValuesText(Series1,Series2,Series3):
    """
    J'utilise cette fonction pour selectionner une valeur parmi trois valeurs à tester 
    """
    values_ = []

    for x, y, z in zip(Series1,Series2,Series3):
        
        if pd.isna(x) & pd.isna(y) & pd.isna(z):
            val = np.nan
        elif pd.isna(x)==False:
            val= x
        elif pd.isna(x):
            s = pd.Series([y,z])
            val = s[s.first_valid_index()]
        values_.append(val)
        
    return values_

In [49]:
# Date_Sortie
date_sortie = __Choose2ValuesText(df_merge['JV_launch_date'],df_merge['MC_Date'])
df_merge['DateSortie'] = date_sortie
df_merge['DateSortie'] = pd.to_datetime(df_merge['DateSortie'])

In [50]:
#Mois de sortie 
df_merge['Mois'] = df_merge['DateSortie'].dt.month

In [51]:
studios = __Choose2ValuesText(df_merge['MC_Studio'],df_merge['GK_developpeur'])
df_merge['Studio'] = studios

In [52]:
#Studio 
df_merge['Score'] = __Choose3ValuesNumber(df_merge['MC_Score'],df_merge['JV_journalist_rate'],df_merge['GK_noteGK'])

In [53]:
#rate
df_merge['Rate'] = __Choose3ValuesNumber(df_merge['JV_players_rates'],df_merge['GK_noteUsers'],df_merge['MC_Userscore'])

In [54]:
#Reviews
df_merge['Review'] = __Choose2ValuesNumber(df_merge['JV_reviews'],df_merge['GK_nbAvis'])

In [55]:
df_final = df_merge[['Name','Platform','Year','Genre','Publisher','DateSortie','Mois','Studio','Score',\
                     'Rate','Review',\
                     'NA_Sales','EU_Sales','JP_Sales','Other_Sales',\
                     'Global_Sales','GK_licence','GK_distributeur']]

In [65]:
gklicenses = []

for lic in df_final['GK_licence']:
    gklicenses.append(str(lic).strip())

df_final['GK_licence'] = gklicenses
df_final = df_final.drop_duplicates()

In [40]:
df_final.to_csv('../data/02_GameSpy_Scraping_MergeAllDataSet.csv',sep=',')